In [2]:
import pandas as pd
import nltk
import numpy as np
import re  #regular expressions
from nltk.stem import wordnet  # for lemmatization
from sklearn.feature_extraction.text import CountVectorizer  # for bag of words (bow)
from sklearn.feature_extraction.text import TfidfVectorizer  #for tfidf
from nltk import pos_tag  # for parts of speech
from sklearn.metrics import pairwise_distances  # cosine similarity
from nltk import word_tokenize
from nltk.corpus import stopwords
import random
nltk.download('omw-1.4')
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\MSI\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MSI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\MSI\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MSI\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MSI\AppData\Roaming\nltk_data...


True

In [1]:
import json

with open('intents.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data['intents'])  #the 'intents' data is a list of dictionaries, the DataFrame will have columns corresponding to the keys of these dictionaries.
df

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 821: character maps to <undefined>

In [2]:
df['responses'][15] #Retrieve the element at index 15 of 'responses' column.

NameError: name 'df' is not defined

##Preprocessing of the data

In [4]:
def format_menu(menu): #reformat the locations 15 , 16 in responses column
    formatted_menu = ""
    for category in menu:
        formatted_menu += f"{category['category']}:\n" #first printing the name of the following category
        for item in category['items']:
            formatted_menu += f"{item['name']} - ${item['price']}\n" #accessing each item in this category write the (item's name - it's price)
    return formatted_menu.strip()

df.loc[14:15, 'responses'] = df.loc[14:15, 'responses'].apply(format_menu) #apply the function format_menu on indices 15 ,16 only of responses column .
df

,tag,patterns,responses
0,greeting_arabic,"[اهلا, مرحبا, ازيك, صباح الخير, السلام عليكم]","[اهلا, ًصباح الخير, أقدر اساعدك ازاي؟]"
1,greeting_english,"[Hi, Hey, Hello, good afternoon, good morning]","[Hello!, Hi, How can I help?, Hello! How can I..."
2,location_arabic,"[فين الأكل, عايز مطعم, مكان المطاعم, فين المطع...",[جامعتنا فيها مطاعم كتير. ممكن تحدد موقعك الحا...
3,location_english,"[Where do I go to eat?, I'm hungry, Where are ...",[There are many places to eat across the campu...
4,restaurant_suggestion_arabic,"[اقترح عليا اكل, اكل ايه؟]",[نفسك تاكل ايه دلوقتي؟ نجرسكو او سوري او بيتزا...
5,restaurant_suggestion_english,"[Suggest somewhere to eat, Where do you think ...",[What kind of food would you like to order so ...
6,pasta_recommendation_arabic,"[نفسي في نجرسكو, مكرونة بشاميل]","[جرب ويتش, بيعمل احسن مكرونة]"
7,pasta_recommendation_english,"[i would like to order pasta, i would like to ...","[You should try Witch, it has the best reviews..."
8,sandwiches_recommendation_arabic,"[نفسي في سندوتش بانيه, ساندوتش كرسبي, برجر]",[المروة عنده سندوتشات لذيذة. هتلاقيه جنب مدرج 17]
9,sandwiches_recommendation_english,"[i'm craving a chicken sandwich, i would like ...","[Then I'd suggest El Marwa, it has the best re..."


In [5]:
print(df['responses'][14])
print(df['responses'][15])

Meals of Witch:
باستا صوص ابيض - $30
سندوتش بطاطس - $22
برجر - $10.99
كريب زنجر - $55
كفتة - $35
بيتزا فراح - $75
بيتزا لحمة - $75
بيتزا مارجريتا - $70
بينزا تونا - $85
بيتزا ميكس - $85
بيتزا ميكس لحمة - $85
desserts of Witch:
مولتن كيك - $45
تشيز كيك - $40
كاب كيك - $30
فطيرة شكولاتة - $35
drinks of Witch:
ايسد كوفي - $10
سموزي - $30
شاي - $10
قهوة - $10
بيبسي - $5
نسكافيه - $15
Meals of Marwa:
باستا صوص ابيض - $30
سندوتش بطاطس - $22
برجر - $10.99
كريب زنجر - $55
كفتة - $35
بيتزا فراح - $75
بيتزا لحمة - $75
بيتزا مارجريتا - $70
بينزا تونا - $85
بيتزا ميكس - $85
بيتزا ميكس لحمة - $85
desserts of Marwa:
مولتن كيك - $45
تشيز كيك - $40
كاب كيك - $30
فطيرة شكولاتة - $35
drinks of Marwa:
ايسد كوفي - $10
سموزي - $30
شاي - $10
قهوة - $10
بيبسي - $5
نسكافيه - $15
Meals of Africanno:
باستا صوص ابيض - $30
سندوتش بطاطس - $22
برجر - $10.99
كريب زنجر - $55
كفتة - $35
بيتزا فراح - $75
بيتزا لحمة - $75
بيتزا مارجريتا - $70
بينزا تونا - $85
بيتزا ميكس - $85
بيتزا ميكس لحمة - $85
desserts of Africanno:

##Makes a DataFrame where each row represents an intent with multiple patterns and responses, and it transforms it into a new DataFrame where each row represents a single pattern-response pair along with its associated tag. This could be useful for data preprocessing or analysis tasks.

In [6]:
dic = {"tag":[], "patterns":[], "responses":[]} #initialize a dictionary
for i in range(len(df)): #iterates over each row of the DataFrame df , it extracts the values of the 'patterns', 'responses', and 'tag' columns for the current row
    ptrns = df[df.index == i]['patterns'].values[0]
    rspns = df[df.index == i]['responses'].values[0]
    tag = df[df.index == i]['tag'].values[0]
    for j in range(len(ptrns)): #it appends the value of current row to the dictionary
        dic['tag'].append(tag)
        dic['patterns'].append(ptrns[j])
        dic['responses'].append(rspns)

df = pd.DataFrame.from_dict(dic) #convert the dicionary into a dataframe to organize the data into a tabular format , that makes the data more easier in further analysis
df

,tag,patterns,responses
0,greeting_arabic,اهلا,"[اهلا, ًصباح الخير, أقدر اساعدك ازاي؟]"
1,greeting_arabic,مرحبا,"[اهلا, ًصباح الخير, أقدر اساعدك ازاي؟]"
2,greeting_arabic,ازيك,"[اهلا, ًصباح الخير, أقدر اساعدك ازاي؟]"
3,greeting_arabic,صباح الخير,"[اهلا, ًصباح الخير, أقدر اساعدك ازاي؟]"
4,greeting_arabic,السلام عليكم,"[اهلا, ًصباح الخير, أقدر اساعدك ازاي؟]"
...,...,...,...
104,africana_location_english,faculty of Science,[Africana is located near the faculty of Scien...
105,africana_location_english,Classes and Labs,[Africana is located near the faculty of Scien...
106,africana_location_english,Hall 18,[Africana is located near the faculty of Scien...
107,thank_you_arabic,شكرا,"[العفو, قولي لو عندك أسئلة تانية]"


In [7]:
def cleaning(x):
    cleaned_array = list()
    for i in x:
        a = str(i).lower()  # convert to all lower letters
        p = re.sub(r'[^a-z0-9]', ' ', a)  # remove any special characters as [# ,& , $ , ...] but keep numbers
        cleaned_array.append(p)  # add variable p to our array names cleaned_array
    return cleaned_array


#Calling the function:
# Create extra column in our dataset to see how the cleaned text looks like in comparison with the original
df.insert(1, 'Cleaned Context', cleaning(df['patterns']), True)
# first argument indicates position we want this column to be placed, second is the name of the new column,
# third is the array we want to you to fill the rows with
# the last boolean indicates whether to allow duplicates
df

,tag,Cleaned Context,patterns,responses
0,greeting_arabic,,اهلا,"[اهلا, ًصباح الخير, أقدر اساعدك ازاي؟]"
1,greeting_arabic,,مرحبا,"[اهلا, ًصباح الخير, أقدر اساعدك ازاي؟]"
2,greeting_arabic,,ازيك,"[اهلا, ًصباح الخير, أقدر اساعدك ازاي؟]"
3,greeting_arabic,,صباح الخير,"[اهلا, ًصباح الخير, أقدر اساعدك ازاي؟]"
4,greeting_arabic,,السلام عليكم,"[اهلا, ًصباح الخير, أقدر اساعدك ازاي؟]"
...,...,...,...,...
104,africana_location_english,faculty of science,faculty of Science,[Africana is located near the faculty of Scien...
105,africana_location_english,classes and labs,Classes and Labs,[Africana is located near the faculty of Scien...
106,africana_location_english,hall 18,Hall 18,[Africana is located near the faculty of Scien...
107,thank_you_arabic,,شكرا,"[العفو, قولي لو عندك أسئلة تانية]"


##Tokenization breaks down text into smaller units, Lemmatization reduces words to their base forms(as present if the word is verb or as singular if the word is noun). Both are important preprocessing steps in NLP

In [8]:
def text_normalization(text):
    text = str(text).lower()  # convert to all lower letters
    spl_char_text = re.sub(r'[^a-z]', ' ', text)  # remove any special characters including numbers
    tokens = nltk.word_tokenize(spl_char_text)  # tokenize words
    lema = wordnet.WordNetLemmatizer()  # lemmatizer initiation
    tags_list = pos_tag(tokens, tagset = None)  # parts of speech
    lema_words = []
    for token, pos_token in tags_list:
        if pos_token.startswith('V'):  # if the tag from tag_list is a verb, assign 'v' to it's pos_val
            pos_val = 'v'
        elif pos_token.startswith('J'):  # adjective
            pos_val = 'a'
        elif pos_token.startswith('R'):  # adverb
            pos_val = 'r'
        else:  # otherwise it must be a noun
            pos_val = 'n'
        lema_token = lema.lemmatize(token, pos_val)  # performing lemmatization
        lema_words.append(lema_token)  # added the lemamtized words into our list
    return " ".join(lema_words)  # return our list as a human sentence

In [11]:
normalized = df['patterns'].apply(text_normalization) #calling the function
df.insert(2, 'Normalized patterns', normalized, True)
df

,tag,Cleaned Context,Normalized patterns,patterns,responses
0,greeting_arabic,,,اهلا,"[اهلا, ًصباح الخير, أقدر اساعدك ازاي؟]"
1,greeting_arabic,,,مرحبا,"[اهلا, ًصباح الخير, أقدر اساعدك ازاي؟]"
2,greeting_arabic,,,ازيك,"[اهلا, ًصباح الخير, أقدر اساعدك ازاي؟]"
3,greeting_arabic,,,صباح الخير,"[اهلا, ًصباح الخير, أقدر اساعدك ازاي؟]"
4,greeting_arabic,,,السلام عليكم,"[اهلا, ًصباح الخير, أقدر اساعدك ازاي؟]"
...,...,...,...,...,...
104,africana_location_english,faculty of science,faculty of science,faculty of Science,[Africana is located near the faculty of Scien...
105,africana_location_english,classes and labs,class and lab,Classes and Labs,[Africana is located near the faculty of Scien...
106,africana_location_english,hall 18,hall,Hall 18,[Africana is located near the faculty of Scien...
107,thank_you_arabic,,,شكرا,"[العفو, قولي لو عندك أسئلة تانية]"


#Removing the stopwords
as any prounouns , conjunctions , common adverbs , articles ,..

In [12]:
stop = stopwords.words('english')
def removeStopWords(text):
    Q = []
    s = text.split()  # create an array of words from our text sentence, cut it into words
    q = ''
    for w in s:  # for every word in the given sentence if the word is a stop word ignore it
        if w in stop:
            continue
        else:  # otherwise add it to the end of our array
            Q.append(w)
        q = " ".join(Q)  # create a sentence out of our array of non stop words
    return q

In [13]:
normalized_non_stopwords = df['Normalized patterns'].apply(removeStopWords)
df.insert(3, 'Normalized and StopWords Removed', normalized_non_stopwords, True)
df

,tag,Cleaned Context,Normalized patterns,Normalized and StopWords Removed,patterns,responses
0,greeting_arabic,,,,اهلا,"[اهلا, ًصباح الخير, أقدر اساعدك ازاي؟]"
1,greeting_arabic,,,,مرحبا,"[اهلا, ًصباح الخير, أقدر اساعدك ازاي؟]"
2,greeting_arabic,,,,ازيك,"[اهلا, ًصباح الخير, أقدر اساعدك ازاي؟]"
3,greeting_arabic,,,,صباح الخير,"[اهلا, ًصباح الخير, أقدر اساعدك ازاي؟]"
4,greeting_arabic,,,,السلام عليكم,"[اهلا, ًصباح الخير, أقدر اساعدك ازاي؟]"
...,...,...,...,...,...,...
104,africana_location_english,faculty of science,faculty of science,faculty science,faculty of Science,[Africana is located near the faculty of Scien...
105,africana_location_english,classes and labs,class and lab,class lab,Classes and Labs,[Africana is located near the faculty of Scien...
106,africana_location_english,hall 18,hall,hall,Hall 18,[Africana is located near the faculty of Scien...
107,thank_you_arabic,,,,شكرا,"[العفو, قولي لو عندك أسئلة تانية]"


##CountVectorizer takes a set of text documents, counts the occurrence of each word (token) in each document, and represents this information as a matrix of token counts

In [28]:
cv = CountVectorizer()  # initializing count vectorizer
x_bow = cv.fit_transform(df['Normalized patterns']).toarray()  # badly speaking this converts words to vectors

features_bow = cv.get_feature_names_out()  # use function to get all the normalized words
df_bow = pd.DataFrame(x_bow, columns = features_bow)  # create dataframe to show the 0, 1 value for each word
df_bow

,about,afternoon,and,any,at,available,be,beef,best,burger,...,want,what,where,which,white,witch,work,would,you,your
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
105,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
106,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##Getting the appropriate response according to [Matching the cosine value of the question with nearest value to it from those of the responses]

In [50]:
def chat_bow(question):
    tidy_question = text_normalization(removeStopWords(question))  # clean & lemmatize the question
    cv_ = cv.transform([tidy_question]).toarray()  # convert the question into a vector
    cos = 1- pairwise_distances(df_bow, cv_, metric = 'cosine')  # calculate the cosine value
    index_value = cos.argmax()  # find the index of the maximum cosine value , getting the most similar index value to the cosine value of the question
    response_value = df['responses'].loc[index_value]  # get the response value
    if isinstance(response_value, list): #if the reterived response is a list of responses , we choose one of them randomly
        selected_response = random.choice(response_value)  # select a random response from the list
    else:
        selected_response = response_value
    print(selected_response)

In [38]:
chat_bow('hi')

which food do you want ?


In [51]:
chat_bow('I want to eat')

There are many places to eat across the campus. What's your current location so I can suggest somwhere nearby?


In [52]:
chat_bow('Where are the university restaurants')

There are many places to eat across the campus. What's your current location so I can suggest somwhere nearby?


In [54]:
chat_bow('Where do you think I should eat?')

What kind of food would you like to order so I could recommend the best place for it? Would you like dishes like white sauce pasta? or pizza and sandwiches? or Would u like to buy snacks?


In [53]:
chat_bow('بكام؟')

اهلا


In [55]:
chat_bow('What are the prices now?')

Meals of Witch:
white sauce pasta - $30
fried potatoes sandwich - $22
burger - $10.99
zenger creb - $55
kofta - $35
chicken pizza - $75
beef pizza - $75
margherita pizza - $70
tuna pizza - $85
pizza mix chicken - $85
pizza mix beef - $85
desserts of Witch:
Chocolate Lava Cake - $45
Cheesecake - $40
cupcake - $30
chocolate pizza - $35
drinks of Witch:
Iced Coffee - $10
any Fruit Smoothie - $30
tea - $10
coffee - $10
cola - $5
latte - $15
Meals of Marwa:
white sauce pasta - $30
fried potatoes sandwich - $22
burger - $10.99
zenger creb - $55
kofta - $35
chicken pizza - $75
beef pizza - $75
margherita pizza - $70
tuna pizza - $85
pizza mix chicken - $85
pizza mix beef - $85
desserts of Marwa:
Chocolate Lava Cake - $45
Cheesecake - $40
cupcake - $30
chocolate pizza - $35
drinks of Marwa:
Iced Coffee - $10
any Fruit Smoothie - $30
tea - $10
coffee - $10
cola - $5
latte - $15
Meals of Africanno:
white sauce pasta - $30
fried potatoes sandwich - $22
burger - $10.99
zenger creb - $55
kofta - $3

In [44]:
chat_bow('price')

Meals of Witch:
white sauce pasta - $30
fried potatoes sandwich - $22
burger - $10.99
zenger creb - $55
kofta - $35
chicken pizza - $75
beef pizza - $75
margherita pizza - $70
tuna pizza - $85
pizza mix chicken - $85
pizza mix beef - $85
desserts of Witch:
Chocolate Lava Cake - $45
Cheesecake - $40
cupcake - $30
chocolate pizza - $35
drinks of Witch:
Iced Coffee - $10
any Fruit Smoothie - $30
tea - $10
coffee - $10
cola - $5
latte - $15
Meals of Marwa:
white sauce pasta - $30
fried potatoes sandwich - $22
burger - $10.99
zenger creb - $55
kofta - $35
chicken pizza - $75
beef pizza - $75
margherita pizza - $70
tuna pizza - $85
pizza mix chicken - $85
pizza mix beef - $85
desserts of Marwa:
Chocolate Lava Cake - $45
Cheesecake - $40
cupcake - $30
chocolate pizza - $35
drinks of Marwa:
Iced Coffee - $10
any Fruit Smoothie - $30
tea - $10
coffee - $10
cola - $5
latte - $15
Meals of Africanno:
white sauce pasta - $30
fried potatoes sandwich - $22
burger - $10.99
zenger creb - $55
kofta - $3

In [58]:
chat_bow('hi')

Hi there! what food do you want ?


In [23]:
chat_bow('شكرا')

ًصباح الخير, أقدر اساعدك ازاي؟
